# Analyse the Extracted Samples

In [5]:
# import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pickle
import warnings
import tqdm
import pandas as pd
import librosa
import librosa.display as display
import biosppy
from scipy import signal
from scipy.stats import zscore
from scipy.io import wavfile
from scipy.fft import fft,fftfreq
from scipy.fftpack import fft

warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib as mlp
mlp.rc("xtick",labelsize=10)
mlp.rc("ytick",labelsize=10)
mlp.rc("axes",labelsize=11)
plt.rcParams["figure.figsize"] = [11,5]
plt.rcParams["figure.dpi"] = 300

CURR_DIR = os.getcwd()

In [6]:
# make necessary definitions

# directories definitions
MAIN_DIR = "."
if os.path.basename(os.getcwd())!="Silent-Interface-for-IOT-Devices":
    os.chdir("..")

DATA_DIR = os.path.join(MAIN_DIR,"dataset")
NEW_DATA_DIR = os.path.join(MAIN_DIR, "new dataset")
FIG_DIR = os.path.join(MAIN_DIR,"figures")
PICKLE_DIR = os.path.join(MAIN_DIR,"pickles")
os.makedirs(FIG_DIR,exist_ok=True)
os.makedirs(PICKLE_DIR,exist_ok=True)

# hardware definitions
"""
https://docs.openbci.com/docs/02Cyton/CytonDataFormat#:~:text=By%20default%2C%20our%20Arduino%20sketch,of%200.02235%20microVolts%20per%20count
"""
SAMPLING_RATE = 250 #Hz
NUM_CHANNELS = 8 
ADC_RESOLUTION = 24 #bits
ADC_GAIN = 24.0
REF_VOLTAGE = 4.5 #Volts
SCALE_FACTOR = (REF_VOLTAGE/float((pow(2,23))-1)/ADC_GAIN)*1000000.0 #micro-volts

# dataset definitions
SPEAKER = ["RL","RN","SR","US"]
SESSION = ["session1","session2","session3"]
MODE = ["mentally","mouthed","audible"]
SENTENCES =["अबको समय सुनाउ","एउटा सङ्गित बजाउ","आजको मौसम बताउ","बत्तिको अवस्था बदल","पङ्खाको स्तिथी बदल"]
WORDS = ["समय","सङ्गित","मौसम","बत्ति","पङ्खा"]

SENTENCE_LABEL= np.array(SENTENCES)[[0,1,2,0,3,1,0,3,0,0,1,1,3,3,4,4,2,3,1,2,2,2,4,4,4]]
WORD_LABEL= np.array(WORDS)[[4,0,3,1,0,1,1,0,4,0,3,2,4,4,2,1,4,1,2,2,2,0,3,3,3]]

NEW_LABEL = np.array(SENTENCES)[[3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0]]

LABELS = {"word":WORD_LABEL,"sentence":SENTENCE_LABEL}

# a function to save plotted figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(FIG_DIR, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    

##### File parser

In [7]:
listOfFiles = glob.glob(NEW_DATA_DIR+"/**/*.txt",recursive=True)
# listOfFiles = glob.glob(NEW_DATA_DIR+"/**/*.txt",recursive=True)
print("no of files", len(listOfFiles))

no of files 104


In [8]:
def parser(files,FILTER=True, maxSignalLength = 0):
    """
    parser function to extract utterances from .txt file and store them in a dictionary
    """
    
    dataset = {"data":[], "speaker":[], "labels":[], "session":[], "filename":[]}

    def get_data(file):
        session = file.split("/")[-2]
        speaker = file.split("/")[-3]
        filename = file.split("/")[-1]
        signal = read_data(file)
#         print(len(signal), "file : ", filename)
        
        # only take files with 15 labels or max labels for utterance in a stream.         
        if(len(signal) != len(NEW_LABEL)) and FILTER :
            return
        
        dataset["data"].extend(signal)
        dataset["speaker"].extend([speaker]*len(signal))
        dataset["session"].extend([session]*len(signal))
        dataset["labels"].extend(NEW_LABEL)
        dataset["filename"].extend([filename]*len(signal))
        
    def read_data(file):
        f = open(file, 'r')
        contents = map(lambda x : x.strip(), f.readlines())
        #the file starts with '%' and some instruction before data and removing these data 
        frames_original = list(filter(lambda x : x and x[0] != '%', contents))[1:]
        #the data row contains channels info digital trigger and accelerometer info separated by comma
        frames_original = list(map(lambda s : list(map( lambda ss: ss.strip(), s.split(','))), frames_original))
        # (8 channels) + digital triggers
        # the digital trigger is in a[16], used to indicate the utterance
        frames = list(map(lambda a: list(map(float, a[1:9])) + [float(a[16])] , frames_original))
        frames = np.array(frames)
        indices = []
        signal = []
        for index,f in enumerate(frames[:,-1]):
            if(bool(f) ^ bool(frames[(index+1) if ((index+1)<len(frames)) else index,-1]) ):
                indices.append(index)
                if len(indices)>1 and len(indices)%2==0:
                    signalTrueLength = indices[len(indices)-1] - indices[len(indices)-2]
                    padWidth = 0
                    if (maxSignalLength != 0) and  (signalTrueLength < maxSignalLength):
                        padWidth = maxSignalLength - signalTrueLength
                    
                    signalStartIndex = indices[len(indices)-2] - int(np.floor(padWidth/2))
                    signalEndIndex = indices[len(indices)-1] + int(np.ceil(padWidth/2))
                    signal.append(frames[signalStartIndex : signalEndIndex, :-1])   
        
        # convert to microVolts and return
        return np.array(signal)*SCALE_FACTOR
        
    for file,i in zip(files,tqdm.tqdm(range(1,len(files)+1),desc="PARSING DATA")):
        get_data(file)
    
    return dataset

###### Raw Data Imports and Export

In [9]:
pickleDataFile =  "new_data_dict.pickle" 
dataNeedsUpdate = True
if (pickleDataFile in os.listdir(PICKLE_DIR)) and (not dataNeedsUpdate):
    #TODO : implement MLTD here ???
    parsedData = pickle.load(open(os.path.join(PICKLE_DIR, pickleDataFile), "rb"))
else : 
    parsedData = parser(listOfFiles)
    pickle.dump(parsedData, open(os.path.join(PICKLE_DIR, pickleDataFile), "wb"))

PARSING DATA:  99%|█████████▉| 103/104 [04:31<00:02,  2.63s/it]


In [10]:
print(type(parsedData))
print(parsedData.keys())
print([len(parsedData[i]) for i in parsedData.keys()])

<class 'dict'>
dict_keys(['data', 'speaker', 'labels', 'session', 'filename'])
[1455, 1455, 1455, 1455, 1455]


#### Raw Dataframe Import and Export 

In [11]:
df_pickleDataFile = 'new_data_dict_dataframe.pickle'
df_dataNeedsUpdate = True
if df_pickleDataFile in os.listdir(PICKLE_DIR) and not df_dataNeedsUpdate:
    df = pd.read_pickle(os.path.join(PICKLE_DIR, df_pickleDataFile))
else :
    df = pd.DataFrame(parsedData)
    df.to_pickle(os.path.join(PICKLE_DIR, df_pickleDataFile))

In [12]:
(df["data"].iloc[0].shape)
df.shape


(1455, 5)

### finding the max length of samples

In [13]:
max([len(df["data"].iloc[i]) for i in range(df.shape[0]) ])

1875

##### Raw Data Import and Export :
##### taking surrounding emg to make the total signal of equal length (to aviod paddings)

In [15]:
maxSignalLength = 1875 #from above
pickleDataFile =  "new_data_dict_equalmaxlen.pickle" 
dataNeedsUpdate = True
if (pickleDataFile in os.listdir(PICKLE_DIR)) and (not dataNeedsUpdate):
    parsedData = pickle.load(open(os.path.join(PICKLE_DIR, pickleDataFile), "rb"))
else : 
    parsedData = parser(listOfFiles, maxSignalLength = maxSignalLength)
    pickle.dump(parsedData, open(os.path.join(PICKLE_DIR, pickleDataFile), "wb"))

PARSING DATA:  99%|█████████▉| 103/104 [04:22<00:02,  2.55s/it]


#### Raw Dataframe Import and Export

In [16]:
df_pickleDataFile = 'new_data_dict_dataframe_equalmaxlen.pickle'
df_dataNeedsUpdate = True
if df_pickleDataFile in os.listdir(PICKLE_DIR) and not df_dataNeedsUpdate:
    df = pd.read_pickle(os.path.join(PICKLE_DIR, df_pickleDataFile))
else :
    df = pd.DataFrame(parsedData)
    df.to_pickle(os.path.join(PICKLE_DIR, df_pickleDataFile))

In [17]:
df["data"].iloc[0]

array([[-296.67374373,  374.01287502,  158.81399014, ...,  677.14299872,
         -41.1475943 ,  155.25884041],
       [-297.82383266,  373.63669341,  157.31869336, ...,  675.75457121,
         -42.79877213,  153.90941933],
       [-300.08651023,  372.53905687,  154.41051095, ...,  672.97981169,
         -45.7938786 ,  150.9707652 ],
       ...,
       [-294.41455862,  375.58513689,  162.47856301, ...,  681.11130647,
         -39.22712871,  160.06199892],
       [-293.57422907,  375.25042825,  163.4587765 , ...,  681.9366621 ,
         -38.5146996 ,  160.93879194],
       [-295.67653724,  374.31514744,  160.72196474, ...,  679.42065021,
         -41.2655003 ,  158.3009041 ]])

In [18]:
[len(df["data"].iloc[i]) for i in range(df.shape[0]) ]

[1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,
 1875,

#### Data Access

In [19]:
def get_sample(index, speaker, label = "आजको मौसम बताउ"):
    return df.query('speaker=="'+speaker+'" and labels=="'+ label+'"')["data"].iloc[index]

In [20]:
print("Length of RL samples in seconds : %.2f s" % (len(get_sample(1, 'RL', SENTENCES[0]))/SAMPLING_RATE))
print("Length of US samples in seconds : %.2f s" % (len(get_sample(1, 'US', SENTENCES[0]))/SAMPLING_RATE))

Length of RL samples in seconds : 7.50 s
Length of US samples in seconds : 7.50 s


#### Data Filtering

In [21]:
def preprocess_data(data):
    #correct dc drift
    
    #correct dc bias
    
    #normalize data
    
    #filter data
    def digital_filter(data,HPF=0.5,LPF=10,H_ORDER=4,L_ORDER=4,SR=SAMPLING_RATE):
        # Highpass filter
        f_signal = biosppy.signals.tools.filter_signal(data,
                                                     ftype="butter",
                                                     band="highpass",
                                                     order=H_ORDER,
                                                     sampling_rate=SR,
                                                     frequency=HPF)

        b,a = signal.iirnotch(50,30,SR)
        f_signal = signal.lfilter(b,a,f_signal[0])
        # Lowpass filter
        f_signal = biosppy.signals.tools.filter_signal(f_signal,
                                                     ftype="butter",
                                                     band="lowpass",
                                                     order=L_ORDER,
                                                     sampling_rate=SR,
                                                     frequency=LPF)
        return f_signal[0]
    
    f_data = []
    for i in range(NUM_CHANNELS):
        c_data = data[:,i]- data[0,i]
        c_data = c_data - np.mean(c_data)
        c_data = digital_filter(c_data)
        f_data.append(c_data)
    
    return np.array(f_data).T

#### Preprocess dataframe import and export

In [23]:
for i in zip(tqdm.tqdm(range(len(df["data"].iloc[:])),desc="PARSING DATA")):
# for i in range(len(df["data"].iloc[:])):
    df["data"].iloc[i] = preprocess_data(df["data"].iloc[i])
    
df_pickleDataFile = 'new_data_dict_dataframe_equalmaxlen_preprocess.pickle'
df_dataNeedsUpdate = True
if df_pickleDataFile in os.listdir(PICKLE_DIR) and not df_dataNeedsUpdate:
    df = pd.read_pickle(os.path.join(PICKLE_DIR, df_pickleDataFile))
else :
    df.to_pickle(os.path.join(PICKLE_DIR, df_pickleDataFile))

PARSING DATA: 100%|██████████| 1455/1455 [01:42<00:00, 14.23it/s]


### Data Feature

In [64]:
class EMG(object):
    """
    preprocessing and feature extraction class for EMG data
    X = List of all instances of input data
    x = an instance of the input data 
    Y = List of all instances of input labels
    y = an instance of the input label
    
    average length of a word utterance: ( 600ms(100 wpm) + 480ms (130 wpm) + 360ms (160 wpm) ) / 3 =  480ms
    """
    
    def __init__(self, SR,FRAME_SIZE,FRAME_SHIFT,MODE):
        """Set the variables """
        self.SR = SR
        self.FRAME_SIZE = FRAME_SIZE
        self.FRAME_SHIFT = FRAME_SHIFT
        self.MODE = MODE
        self.pickle_name = "X_"+MODE+"(features).pickle"
    

    def DNPA(self,seg):
        """Double Nine Point Average"""
        w = []
        for i in range(len(seg)):
            a = i - 4
            b = i + 4
            a = a if a>=0 else 0
            b = b if b<len(seg) else len(seg)
            w.append(int(np.sum(seg[a:b])/9))

        v = []
        for i in range(len(seg)):
            a = i - 4
            b = i + 4
            a = a if a>=0 else 0
            b = b if b<len(seg) else len(seg)
            v.append(int(np.sum(w[a:b])/9))

        return v

    
    def ZCR(self,seg):
        """Zero Crossing Rate"""
        pos = seg>0
        npos = ~pos
        return len(((pos[:-1] & npos[1:]) | (npos[:-1] & pos[1:])).nonzero()[0])

    
    def HFS(self,seg):
        """High frequency signals"""
        return np.subtract(seg,self.DNPA(seg))

    
    def RHFS(self,seg):
        """Rectified High frequency signals"""
        return abs(self.HFS(seg))

    
    def FBP(self,seg):
        """Frame Based Power"""
        return np.sum(np.power(seg,2))

    
    def feature(self,seg,type="time"):
        """ 
        "time": Features in time domain
        "freq": Features in frequency domain
        "all": Features in both domain
        """
        if type == "time":
            return np.hstack((self.DNPA(seg),self.RHFS(seg),self.HFS(seg),self.ZCR(seg),self.FBP(seg)))
        elif type == "freq":
            return 
        elif type == "all":
            return np.hstack(self.feature(seg,"time"),self.feature(seg,"freq"))

    
    def MFCC(self,seg):
        """Mel Frequency Cepstral Coefficients"""
        mfcc = librosa.feature.mfcc(y=seg,sr=self.SR,n_mfcc=20)
#         return np.mean(mfcc.T,axis=0)
        return mfcc

    
    def STFT(self,seg):
        """Short Time Fourier Transform"""
        stft = librosa.feature.chroma_stft(y=seg,sr=self.SR,n_fft=20)
#         return np.mean(stft.T,axis=0)
        return stft

    
    def segment(self,x):
        """Segmenting the data into frames and sliding them according to the frame shift"""
        f = []
        for channel in range(NUM_CHANNELS):
            for i in range(x.shape[0]):
                a = i*self.FRAME_SHIFT
                b = a + self.FRAME_SIZE
                if(b>len(x[0])):
                    break
#                 seg = x[channel][a:b]
                seg = x[a:b, channel]
                f.extend(self.feature(seg))
        return f

    def fit(self,X,Y):
        """Extract Features and return the zero padded list of features"""
        if(self.pickle_name in os.listdir() ):
            print("Fetching Pickle file")
            temp_X = pickle.load(open(self.pickle_name,"rb"))
        else:
            temp_X = []
            for x,count in zip(X, tqdm.tqdm(range(len(X)),ncols=100,desc="Extracting Features("+self.MODE+")" ) ):
                temp_X.append(self.segment(x))

            # save all extracted features to a pickle file
            print("Saving features as: ",self.pickle_name)
            pickle.dump(temp_X,open(self.pickle_name,"wb"))
        temp_X = np.array(temp_X)
        print(len(temp_X))
        print(len(Y))
        print(temp_X.shape)
        return self.reduce_dimension(temp_X),self.encode_labels(Y)

    
    def fit_transform(self,X,Y):
        return self.fit(X,Y)
    
    def getFeature(self, X):
        print(X.shape)
        temp_X = []
        for x,count in zip(X, tqdm.tqdm(range(len(X)),ncols=100,desc="Extracting Features("+self.MODE+")" ) ):
            temp_X.append(self.segment(x))
        temp_X = np.array(temp_X)
        print(len(temp_X))
#         print(len(Y))
        print(temp_X.shape)
        return temp_X


In [65]:
features = EMG(SAMPLING_RATE,16,1,"time")
# XXX,YYY = features.fit_transform()
aa = features.getFeature(df["data"].iloc[:])

Extracting Features(time):  11%|███▌                           | 165/1455 [00:00<00:00, 1625.35it/s]

(1455,)
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in

Extracting Features(time):  45%|█████████████▊                 | 651/1455 [00:00<00:00, 1855.52it/s]


in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segmen

Extracting Features(time): 100%|█████████████████████████████▉| 1454/1455 [00:00<00:00, 2440.99it/s]


in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segment
in segmen

In [53]:
df["data"].iloc[1].shape

(1875, 8)

In [63]:
aa[0]

array([], dtype=float64)

In [28]:
len(df.query('speaker=="RL"'))

720